### Optimal Statistic analysis

In this notebook we're going to explore how to assess the presence of a correlated gravitational wave signal in a fast, frequentist way

(We're going to excuse the fact that this is a frequentist analysis because it's very very useful)

We do this by calculating something called the Optimal Statistic, which provides a measure of the correlations of the pulsar timing residuals as a function of the angular separation of each pulsar pair.

## Load what we need!

In [1]:
import numpy as np
import pickle
import json
import glob
import os

import matplotlib.pyplot as plt

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as eparameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
from enterprise.signals import gp_bases
import enterprise.constants as const
from enterprise_extensions.chromatic.solar_wind import solar_wind, createfourierdesignmatrix_solar_dm
import enterprise_extensions
from enterprise_extensions import models, model_utils, hypermodel, blocks
from enterprise_extensions import timing
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
import time
import h5py
import dill

Optional acor package is not installed. Acor is optionally used to calculate the effective chain length for output in the chain file.


In [2]:
by_backend = selections.Selection(selections.by_backend)

#### Collect the data!

This is a very simple simulated data set that has a loud gravitational wave signal injected into it

(On the order of $\log_{10}\mathrm{A_{GW}} \sim -14.1$)

In [9]:
datadir = "./OS_data_dir/"
parfiles=sorted(glob.glob(datadir+'*.par'))
timfiles=sorted(glob.glob(datadir+'*.tim'))

#parfiles=sorted(glob.glob(datadir+'*.par'))[::2]
#timfiles=sorted(glob.glob(datadir+'*.tim'))[::2]

Load the PTA as pulsar objects

This is a very expensive process and there is a chance it may actually crash the notebook

If this happens, rerun it - if this happens more than three times, uncomment the code in the cell above

In [4]:
psrs=[]
for p, t in zip(parfiles,timfiles):

    print(p)
    psr=Pulsar(p,t,ephem="DE440")
    print(psr)
    psrs.append(psr)

./OS_data_dir/J0030+0451.par
./OS_data_dir/J0101-6422.par
[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 


<enterprise.pulsar.Tempo2Pulsar object at 0x7fa1146a7b50>[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.

./OS_data_dir/J0125-2327.par


./OS_data_dir/J0437-4715.par
./OS_data_dir/J0610-2100.par


./OS_data_dir/J0613-0200.par
./OS_data_dir/J0614-3329.par


./OS_data_dir/J0636-3044.par


...

./OS_data_dir/J2234+0944.par


./OS_data_dir/J2236-5527.par


./OS_data_dir/J2241-5236.par


./OS_data_dir/J2317+1439.par
./OS_data_dir/J2322+2057.par


./OS_data_dir/J2322-2650.par


In [ ]:
noisedict={}
noisefile = './noise_files/MPTA_WN_models_full.json'
with open(noisefile, 'r') as f:
    noisedict=json.load(f)

Below, we're going to initialise the signal models for the pulsars

I've included everything that would standardly be used in a normal analysis as useful future referencing for everyone

The data we are working with only possesses the gravitational wave signal, so we don't need to worry about these other effects that we would usually model

In [ ]:
Tspan=model_utils.get_tspan(psrs)

efac = eparameter.Constant(1)
equad = eparameter.Constant()
ecorr = eparameter.Constant()




In [ ]:
crn = blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, components=30,gamma_val=4.33, name='gw')


In [ ]:
def dm_noise(log10_A,gamma,Tspan,components=30,option="powerlaw"):
    """
    A term to account for stochastic variations in DM. It is based on spin
    noise model, with Fourier amplitudes depending on radio frequency nu
    as ~ 1/nu^2.
    """
    nfreqs = 30
    if option=="powerlaw":
      pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)

    dm_basis = utils.createfourierdesignmatrix_dm(nmodes = components,
                                                  Tspan=Tspan)
    dmn = gp_signals.BasisGP(pl, dm_basis, name='dm_gp')

    return dmn

@signal_base.function
def chrom_yearly_sinusoid(toas, freqs, log10_Amp, phase, idx):
    """
    Chromatic annual sinusoid.
    :param log10_Amp: amplitude of sinusoid
    :param phase: initial phase of sinusoid
    :param idx: index of chromatic dependence
    :return wf: delay time-series [s]
    """

    wf = 10**log10_Amp * np.sin(2 * np.pi * const.fyr * toas + phase)
    return wf * (1400 / freqs) ** idx


In [ ]:
models = []
for pulsar in psrs:

    s = gp_signals.TimingModel()

    #if pulsar.name+"_KAT_MKBF_log10_tnequad" in noisedict.keys():
    #    eq = white_signals.TNEquadNoise(log10_tnequad=equad, selection=by_backend)
    #    s += eq
    ef = white_signals.MeasurementNoise(efac=efac, selection=by_backend)
    s += ef
    
    #if pulsar.name+"_basis_ecorr_KAT_MKBF_log10_ecorr" in noisedict.keys():
    #    #ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=by_backend)
    #    ec = gp_signals.EcorrBasisModel(log10_ecorr=ecorr,selection=by_backend)
    #    s += ec
    s += crn

    models.append(s(pulsar))


In [ ]:
pta = signal_base.PTA(models)

#pta.set_default_params(noisedict)

In [ ]:
print(pta.summary())

Here, we begin to initialise the optimal statistic models

We standardly test 3 different correlated signals

HD: The expected signal of the gravitational waves in the form of the Hellings-Downs correlation, representing the quadrupolar waveform of the gravitational wave

Dipole: A signal that varies in amplitude from totally correlated, to totally anti-correlated across the angular separation range of 0 - 180 degrees

Monopole: A signal that is equally represented in amplitude across the entire angular separation range

Bonus fun: What do you think could create these other correlations? Think about what they mean in a practical sense

In [ ]:
ostat = opt_stat.OptimalStatistic(psrs, pta=pta, orf='hd', bayesephem=False)
ostat_dip = opt_stat.OptimalStatistic(psrs, pta=pta, orf='dipole',bayesephem=False)
ostat_mono = opt_stat.OptimalStatistic(psrs, pta=pta, orf='monopole',bayesephem=False)

Read in our predefined value of the gravitational wave signal:

In [ ]:
with open('./noise_files/OS_MPTA_noise_values.json', "r") as f:
    ml_params = json.load(f)

Compute the optimal statistic!

In [ ]:
xi, rho, sig, OS, OS_sig = ostat.compute_os(params=ml_params)
print(OS, OS_sig, OS/OS_sig)

_, _, _, OS_dip, OS_sig_dip = ostat_dip.compute_os(params=ml_params)
print(OS_dip, OS_sig_dip, OS_dip/OS_sig_dip)

_, _, _, OS_mono, OS_sig_mono = ostat_mono.compute_os(params=ml_params)
print(OS_mono, OS_sig_mono, OS_mono/OS_sig_mono)

Now we visualise!

Numbers mean nothing unless they make sense, let's see if the correlations as a function of angular separation make sense!

In [ ]:
idx=np.argsort(xi)

xi_sorted = xi[idx]
rho_sorted = rho[idx]
sig_sorted = sig[idx]


npsr=np.shape(psrs)[0]

gof=np.zeros(npsr)
rho2=np.zeros(npsr)
sig2=np.zeros(npsr)

In [ ]:
def get_HD_curve(zeta):
    coszeta=np.cos(zeta*np.pi/180.)
    xip=(1.-coszeta)/2.
    HD=3.*(1./3.+xip*(np.log(xip) -1./6.))

    return HD/2.

def weightedavg(rho,sig):
    weights, avg=0.,0.
    for r,s in zip(rho,sig):
        weights += 1./(s*s)
        avg += r/(s*s)

    return avg/weights, np.sqrt(1./weights)


In [ ]:
npairs = 450

xi_mean = []
xi_err = []
rho_avg = []
sig_avg = []

i=0

while i < len(xi_sorted):
    xi_mean.append(np.mean(xi_sorted[i:npairs+i]))
    xi_err.append(np.std(xi_sorted[i:npairs+i]))

    r,s=weightedavg(rho_sorted[i:npairs+i],sig_sorted[i:npairs+i])
    rho_avg.append(r)
    sig_avg.append(s)

    i+= npairs

xi_mean=np.array(xi_mean)
xi_err=np.array(xi_err)

fig,ax = plt.subplots(figsize=(12,7))
#plt.figure(figsize=(10,7))
(_, caps, _) = ax.errorbar(xi_mean*180/np.pi, rho_avg, xerr=xi_err*180/np.pi, yerr=sig_avg, marker='o', ls='', 
                                    color='#003f5c', fmt='o', capsize=7.5, elinewidth=3.2, label = "MPTA interpulsar correlations", zorder=10)
zeta=np.linspace(0.01,180,100)

HD=get_HD_curve(zeta)
plt.title("OS: {0:.2E}; SNR: {1:.2f}".format(OS, OS/OS_sig))
ax.plot(zeta, OS*HD, ls='--', label='Hellings-Downs', color='#117733', lw=3, zorder=1)
plt.plot(zeta, zeta*0.0+OS_mono, ls='--', label='Monopole', color='C1', lw=1.5)
plt.plot(zeta, OS_dip*np.cos(zeta*np.pi/180), ls='--', label='Dipole', color='C2', lw=1.5)

ax.set_xlim(0, 180)
#plt.ylim(-1e-29, 1e-29)
ax.set_ylabel(r'$\hat{A}^2 \Gamma(\zeta_{ab})$', fontsize=15)
ax.set_xlabel(r'$\zeta$ ($^{\circ}$)', fontsize=15)
ax.legend(loc=1, fontsize=15)
#plt.tick_params(labelsize=12, which="both")
ax.tick_params(labelsize=15, which="both")
ax.yaxis.offsetText.set_fontsize(15)
#plt.show()
fig.tight_layout()